In [59]:
#%%
#!pip3 install -r requirements.txt
#import dropbox
from selenium.webdriver.common.keys import Keys
# auxiliary functions modified by Luis.
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime

# %% Starting code
dfolder = 'out'
link = 'https://www.booking.com/'
profile, browser, download_path = kzd.start_up(dfolder, link, download=True) 

#%% Main Page
place = 'Barcelona'
search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
search1.send_keys(place)

# kzd.check_and_click(
#     browser, '.sb-searchbox__button', type='css')
kzd.check_and_click(browser,'//button[@type="submit"]', type='xpath')
print('main search button clicked')

def click(browser,element_type, element):
    thing = browser.find_element(element_type, element)
    browser.execute_script("arguments[0].click()", thing)

main search button clicked


In [60]:
# #%% Search Page
# time.sleep(2)
# dates = browser.find_elements('xpath',
#     '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')
# print('dates: ', len(dates))

In [61]:
#%%
time.sleep(2)

# check_in_date = "2023-05-25"
# check_out_date = "2023-05-28"
# week_before = [check_in_date, check_out_date]
check_in_date = "2023-05-29"
check_out_date = "2023-06-01"
week=[check_in_date, check_out_date]

today_month = datetime.now().month
while today_month < int(week[0].split('-')[1]):
    time.sleep(2)
    dates = browser.find_elements('xpath','//table[@class="aadb8ed6d3"]/tbody/tr/td/span')
    print('dates: ', len(dates))
    kzd.check_and_click(browser, '//button[@class="fc63351294 a822bdf511 e3c025e003 fa565176a8 cfb238afa1 ae1678b153 c9fa5fc96d be298b15fa"]', type='xpath')
    print('month clicked')
    today_month+=1




dates:  59
month clicked
dates:  61
month clicked
dates:  61
month clicked


In [62]:
#%%
def scrollDown(driver, numberOfScrollDowns):
    body = driver.find_element(By.TAG_NAME, "body")
    while numberOfScrollDowns >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        numberOfScrollDowns -= 1
    return driver

#%%
scrollDown(browser, 0.05)


#enter start-date
click(browser, 'xpath', '//span[@data-date="{}"]'.format(week[0]))
#click empty space
click(browser, 'xpath', '//input[@class="ce45093752"]')
#click end date button
click(browser, 'xpath', '//button[@data-testid="date-display-field-end"]')
#select specific end date
click(browser, 'xpath', '//span[@data-date="{}"]'.format(week[1]))

# click on empty space
click(browser, 'xpath', '//input[@class="ce45093752"]')

time.sleep(1)

# click on empty space
kzd.check_and_click(
    browser, '//input[@class="ce45093752"]', type='xpath')
print('blank space clicked')

time.sleep(1)

kzd.check_and_click(browser,'//button[@type="submit"]', type='xpath')
print('search button clicked')

time.sleep(1)

blank space clicked
search button clicked


In [66]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    time.sleep(1)
    return(total_pages)

pages = get_number_pages(browser)
print('total pages: ', pages)
#%%

def BookingReport(deal_boxes):
    page_report = []
    for deal_box in deal_boxes:
        hotel_name = deal_box.find_element(By.CSS_SELECTOR,
            'div[data-testid="title"]'
        ).get_attribute('innerHTML').strip()
        
        if not deal_box.find_elements(By.CSS_SELECTOR, 'span[data-testid="price-and-discounted-price"]'):
            hotel_price = deal_box.find_element(By.CSS_SELECTOR,
                'div[data-testid="price-and-discounted-price"]'
            ).find_element(By.TAG_NAME, 'div').get_attribute('innerHTML').strip()
        else:
            hotel_price = deal_box.find_element(By.CSS_SELECTOR,
                'span[data-testid="price-and-discounted-price"]'
            ).get_attribute('innerHTML').strip()
            hotel_price = hotel_price.replace("€&nbsp;", "")
            hotel_price = hotel_price.replace(",", "")
            hotel_price = int(hotel_price)
        
        try:
            hotel_score = deal_box.find_element(By.CSS_SELECTOR,
                'div[aria-label*="Scored"]'
            ).get_attribute('innerHTML').strip()
        except:
            hotel_score = 'nan'
        page_report.append(
            [hotel_name, hotel_price, hotel_score]
        )
        # print([hotel_name, hotel_price, hotel_score])
    return page_report

#%%        
# Loop pages
full_report = []

for page in range(pages):
    # Get the list of hotel elements on the page

    print('Scraping page {} out of {}...'.format(page+1, pages))
    print('before: ', len(full_report))

    browser.implicitly_wait(2)
    
    deal_boxes = browser.find_elements(
            By.XPATH,
            "//div[@data-testid='property-card']"
            )
    report = BookingReport(deal_boxes)
    time.sleep(4)
    
    full_report.extend(report)
    
    print('after: ', len(full_report))

    scrollDown(browser, 1.5)

   
    kzd.check_and_click(browser, 
        '//button[@aria-label="Next page"]', 
        type='xpath' )
    
    time.sleep(4)

report_df = pd.DataFrame(full_report)

# %%
deal_boxes = browser.find_elements(
            By.XPATH,
            "//div[@data-testid='property-card']"
            )

report = BookingReport(deal_boxes)


total pages:  40
Scraping page 1 out of 40...
before:  0
after:  26
Scraping page 2 out of 40...
before:  26
after:  51
Scraping page 3 out of 40...
before:  51
after:  76
Scraping page 4 out of 40...
before:  76
after:  101
Scraping page 5 out of 40...
before:  101
after:  126
Scraping page 6 out of 40...
before:  126
after:  151
Scraping page 7 out of 40...
before:  151
after:  176
Scraping page 8 out of 40...
before:  176
after:  201
Scraping page 9 out of 40...
before:  201
after:  226
Scraping page 10 out of 40...
before:  226
after:  251
Scraping page 11 out of 40...
before:  251
after:  276
Scraping page 12 out of 40...
before:  276
after:  301
Scraping page 13 out of 40...
before:  301
after:  326
Scraping page 14 out of 40...
before:  326
after:  351
Scraping page 15 out of 40...
before:  351
after:  376
Scraping page 16 out of 40...
before:  376
after:  401
Scraping page 17 out of 40...
before:  401
after:  426
Scraping page 18 out of 40...
before:  426
after:  451
Scraping p

In [1]:
report_df = pd.DataFrame(full_report, columns=['hotel','price','rating'])
report_df['start_date'] = week[0]
report_df['city']=place
i=0
for r in df['price']:
    if type(r) is int:
        pass
    else:
        df['price'][i] = df['price'][i].replace("Price €&nbsp;", "")
        df['price'][i] = df['price'][i].replace(",", "")
        df['price'][i] = int(df['price'][i])
    i+=1
df.to_csv('scrape{}v2.csv'.format(place+week[0]))